In [1]:
from river import compose, preprocessing, ensemble
from river import stream

In [2]:
import pandas as pd
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
def preprocess(df):

    df_labels = pd.read_csv('train_labels.csv')
    
    missing_values = df.isnull().sum()
    percentage_missing = (missing_values / df.shape[0]) * 100
    columns_to_keep = percentage_missing[percentage_missing < 50].index
    
    df = df[columns_to_keep]
    
    df = pd.merge(df,df_labels,on='customer_ID')
    df = df.drop(columns=['customer_ID','S_2'])

    
    int_mean = df.select_dtypes(include=['int64']).mean()
    float_median = df.select_dtypes(include=['float64']).median()
    mode = df.select_dtypes(include=['object']).mode().iloc[0]

    
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].fillna(int_mean[col])
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].fillna(float_median[col])
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].fillna(mode[col])


    df['D_63'] = df['D_63'].map(df['D_63'].value_counts(normalize=True))
    df['D_64'] = df['D_64'].map(df['D_64'].value_counts(normalize=True))

    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['target']),df['target'],test_size=0.2,
                                                        random_state=42)

    model = ensemble.RandomForestClassifier()

    # Train the model with the initial number of trees
    model.learn_one(X_train, y_train)
    
    y_pred = model.predict_one(X_test)

    print(classification_report(y_test,y_pred))

In [ ]:
data_stream = stream.iter_pandas(pd.read_csv('train_data.csv'))

for x in data_stream:
    preprocess(pd.DataFrame([x])).iloc[0].to_dict()